In [1]:
import tensorflow as tf
from tensorflow.keras import preprocessing

In [2]:
samples = ['너 오늘 이뻐 보인다', 
           '나는 오늘 기분이 더러워', 
           '끝내주는데, 좋은 일이 있나봐', 
           '나 좋은 일이 생겼어', 
           '아 오늘 진짜 짜증나', 
           '환상적인데, 정말 좋은거 같아']

labels = [[1], [0], [1], [1], [0], [1]]

In [3]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)

word_index = tokenizer.word_index

In [4]:
EPOCH = 100

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((sequences, labels))
    dataset = dataset.repeat(EPOCH)
    dataset = dataset.batch(1)
    dataset = dataset.shuffle(len(sequences))
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

In [5]:
VOCAB_SIZE = len(word_index) +1
EMB_SIZE = 128

In [6]:
def model_fn(features, labels, mode):
    
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT
    
    embed_input = tf.keras.layers.Embedding(VOCAB_SIZE, EMB_SIZE)(features)
    embed_input = tf.reduce_mean(embed_input, axis=-1)
    
    hidden_layer = tf.keras.layers.Dense(128, activation=tf.nn.relu)(embed_input)
    output_layer = tf.keras.layers.Dense(1)(hidden_layer)
    output = tf.nn.sigmoid(output_layer)
    
    loss = tf.losses.mean_squared_error(output, labels)

    if TRAIN:
        global_step = tf.train.get_global_step()
        train_op = tf.train.AdamOptimizer(1e-3).minimize(loss, global_step)

        return tf.estimator.EstimatorSpec(
                  mode=mode,
                  train_op=train_op,
                  loss=loss)

In [7]:
DATA_OUT_PATH = './data_out/'

import os

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

estimator = tf.estimator.Estimator(model_fn = model_fn, model_dir = DATA_OUT_PATH + 'checkpoint/dnn')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './data_out/checkpoint/dnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001715D412828>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
estimator.train(train_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./data_out/checkpoint/dnn\model.ckpt.
INFO:tensorflow:loss = 0.2499209, step = 1
INFO:tensorflow:global_step/sec: 350.612
INFO:tensorflow:loss = 0.14583096, step = 101 (0.289 sec)
INFO:tensorflow:global_step/sec: 651.086
INFO:tensorflow:loss = 0.018592682, step = 201 (0.151 sec)
INFO:tensorflow:global_step/sec: 835.57
INFO:tensorflow:loss = 0.0209716, step = 301 (0.121 sec)
INFO:tensorflow:global_step/sec: 842.582
INFO:tensorflow:loss = 0.0022858891, step = 401 (0.119 sec)
INFO:tensorflow:global_step/sec: 954.924
INFO:tensorflow:loss = 0.0025552053, step = 501 (0.104 sec)
INFO:tensorflow:Saving checkpoints for 600 into ./data_out/checkpoint/dnn\model.ckpt.
INFO:tensorflow:Loss for final step: 0.0002982992.
